In [1]:
import pandas as pd
from tqdm import tqdm
from convokit import Corpus, Speaker, Utterance, download
from collections import defaultdict
import numpy as np

In [2]:
df = pd.read_csv('data/kokil dec 6 reprepare/conf_good_stripped.csv')
df.head(5)

,HITId,Input.sentence_id,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,Input.speaker_intention,...,Input.deception_quadrant,Input.num_words,Input.num_characters,Input.sno,Input.sno1,Input.similar_text_id,affcon_gamemove,affcon_reasoning,affcon_rapport,affcon_shareinformation
0,301KG0KX9CLR06T6MC6UVPAHBC92HU,22056,Game7-turkey-austria,Train,Game7-turkey-austria-9,197,Im moving my fleet to Alb not for Greece but f...,austria-Game7,Game7-turkey-austria-8,Truth,...,Straightforward,12,56,32,16.0,104,1.0,1.0,1.0,1.0
1,301KG0KX9CLR06T6MC6UVPAHBCAH2A,6906,Game11-austria-italy,Validation,Game11-austria-italy-5,45,"And yes I would like peace on our front, I cou...",austria-Game11,Game11-austria-italy-4,Truth,...,Straightforward,31,132,46,22.0,18629,1.0,1.0,1.0,1.0
2,301KG0KX9CLR06T6MC6UVPAHBCC2HX,3066,Game1-england-germany,Train,Game1-england-germany-271,1468,"okay...well, as the person who has ever seen a...",germany-Game1,Game1-england-germany-270,Truth,...,Straightforward,15,86,79,33.0,8855,1.0,1.0,1.0,1.0
3,301KG0KX9CLR06T6MC6UVPAHBCCH2C,24093,Game9-italy-germany,Train,Game9-italy-germany-70,1460,I think the best thing we can do to keep the a...,germany-Game9,Game9-italy-germany-69,Truth,...,Straightforward,22,104,7,36.0,9729,1.0,1.0,1.0,1.0
4,301KG0KX9CLR06T6MC6UVPAHBCD2HY,1591,Game1-england-italy,Train,Game1-england-italy-273,1809,We'll see if I can keep it friendly.,england-Game1,Game1-england-italy-272,Truth,...,Straightforward,8,36,58,39.0,19784,1.0,1.0,1.0,1.0


In [4]:
speakers_list = df['Input.speaker'].tolist()

In [5]:
corpus_speakers = {k: Speaker(id=k) for k in speakers_list}

In [8]:
corpus = Corpus(filename=download("diplomacy-corpus"))

Dataset already exists at C:\Users\lynne\.convokit\downloads\diplomacy-corpus


In [4]:
corpus.print_summary_stats()

Number of Speakers: 83
Number of Utterances: 17289
Number of Conversations: 246


In [6]:
msg_ids = df['Input.msg_id'].tolist()

In [9]:
corpus_filter = corpus.filter_utterances_by(lambda utt: utt.id in msg_ids)

In [21]:
corpus_filter.print_summary_stats()

Number of Speakers: 83
Number of Utterances: 3474
Number of Conversations: 217


In [10]:
for idx, row in df.iterrows():
    utt = corpus_filter.get_utterance(row['Input.msg_id'])
    utt.meta['affcon_gamemove'] = row['affcon_gamemove']
    utt.meta['affcon_reasoning'] = row['affcon_reasoning']
    utt.meta['affcon_rapport'] = row['affcon_rapport']
    utt.meta['affcon_shareinformation'] = row['affcon_shareinformation']

In [11]:
corpus_filter.dump(r'C:\Users\lynne\Documents\diplomacy-betrayal\data\kokil dec 6 reprepare\convokit_with_affcon')

In [12]:
from convokit import PolitenessStrategies

## POLITENESS STRATEGIES 

In [15]:
ps = PolitenessStrategies(verbose=1000)
corpus_filter = ps.transform(corpus_filter)

In [16]:
utterance_ids = corpus_filter.get_utterance_ids()
rows = []
for uid in utterance_ids:
    rows.append(corpus_filter.get_utterance(uid).meta["politeness_strategies"])
politeness_strategies = pd.DataFrame(rows, index=utterance_ids)
politeness_strategies['msg_id'] = politeness_strategies.index

In [17]:
politeness_strategies.head()

,feature_politeness_==Please==,feature_politeness_==Please_start==,feature_politeness_==HASHEDGE==,feature_politeness_==Indirect_(btw)==,feature_politeness_==Hedges==,feature_politeness_==Factuality==,feature_politeness_==Deference==,feature_politeness_==Gratitude==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,...,feature_politeness_==2nd_person==,feature_politeness_==2nd_person_start==,feature_politeness_==Indirect_(greeting)==,feature_politeness_==Direct_question==,feature_politeness_==Direct_start==,feature_politeness_==HASPOSITIVE==,feature_politeness_==HASNEGATIVE==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==INDICATIVE==,msg_id
Game1-italy-germany-0,0,0,1,0,1,1,0,0,0,0,...,1,0,0,1,1,1,1,0,0,Game1-italy-germany-0
Game1-italy-germany-9,0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,1,1,0,0,Game1-italy-germany-9
Game1-italy-germany-10,0,0,1,0,1,0,0,0,0,1,...,1,0,0,0,1,1,0,0,0,Game1-italy-germany-10
Game1-italy-germany-11,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,1,1,0,0,Game1-italy-germany-11
Game1-italy-germany-13,0,0,1,0,1,0,0,0,0,0,...,1,0,0,0,1,1,1,0,0,Game1-italy-germany-13


In [18]:
politeness_strategies.to_csv('./data/kokil dec 6 reprepare/politeness_strategies.csv')

In [19]:
df_politeness_merge = politeness_strategies.merge(df, left_on='msg_id', right_on='Input.msg_id')

In [20]:
df_politeness_merge.to_csv('./data/kokil dec 6 reprepare/affcon_final_politeness_strategies_merge.csv')

### Finding ngrams 

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.util import ngrams
import nltk, collections
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
full_df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_politeness_strategies_merge.csv')

In [26]:
items = ['affcon_rapport', 'affcon_shareinformation', 'affcon_reasoning', 'affcon_gamemove']

In [24]:
def get_freq_df(item_df):
    word_vectorizer = CountVectorizer(ngram_range=(1,4), analyzer='word')
    sparse_matrix = word_vectorizer.fit_transform(item_df['Input.full_text'])
    frequencies = sum(sparse_matrix).data
    df_frequency = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
    return df_frequency

In [29]:
for item in items: 
    print(item)
    df_yes = full_df[full_df[item] == 1]
    df_no = full_df[full_df[item] == 0]
    
    df_yes_freq = get_freq_df(df_yes)
    df_no_freq = get_freq_df(df_no)
    
    df_yes_freq = df_yes_freq.sort_values(by=['frequency'], ascending=False)
    print(df_yes_freq.head(20))
    
    df_no_freq = df_no_freq.sort_values(by=['frequency'], ascending=False)
    print(df_no_freq.head(20))
    
    df_yes_freq.to_csv('./ngrams_results/kokil dec 6 reprepare/df_' + item +'_yes_ngrams.csv')
    df_no_freq.to_csv('./ngrams_results/kokil dec 6 reprepare/df_' + item +'_no_ngrams.csv')

affcon_rapport
                         frequency
more tempting if              2437
more strength                 1760
more interested               1401
morning but might be          1212
mos arm                        889
mood for message               788
more year it will              648
more in the                    617
more inclined to               599
mos bud gal                    595
moscow offers                  572
morning so didnt think         555
mood                           531
more organized than the        484
mos northward                  467
more organized than            447
more or                        427
most two units                 420
most natural                   381
mos                            370
                      frequency
move straight               390
move out of my              281
multiple dots off of        225
moves and talks             188
multiple dots off           128
moves over the next         125
means yes                 

In [28]:
df_rapport_frequency.to_csv('./ngrams_results/df_shareinfo_ngrams.csv')

### Finding Pronouns

In [30]:
first_person = ["i", "me", "mine", "myself", "we", "us", "ours", "ourselves", "our"]
second_person = ["you","your","yours","yourself","yourselves"]
third_person = ["he", "she", "him", "her", "his", "hers", "himself", "herself"
                "they", "them", "theirs", "themselves", "their"]

In [31]:
items = ['affcon_rapport', 'affcon_shareinformation', 'affcon_reasoning', 'affcon_gamemove']

In [32]:
import re 

def get_pronouns_per_row(item_df):
    first_list = []
    second_list = []
    third_list = []

    total_words = 0
    
    for idx, row in item_df.iterrows():
        text = row['Input.full_text'].strip()
        text_split = text.split(' ')
        for word in text_split:
            word = re.sub("\.|\,|\:|\!|\?|\"|\'|\;|\)|\(", "", word)
            total_words += 1
            word = word.lower()

            if word in first_person: 
                first_list.append(word)
            elif word in second_person: 
                second_list.append(word)
            elif word in third_person:
                third_list.append(word)

    length_first = len(first_list)
    length_second = len(second_list)
    length_third = len(third_list)

    nominalize = 100

    length_first = (length_first/total_words) * nominalize
    length_second = (length_second/total_words) * nominalize
    length_third = (length_third/total_words) * nominalize

    #print('first: ', length_first, '\n', 'second: ', length_second, '\n', 'third: ', length_third)
    
    return length_first, length_second, length_third

In [33]:
for item in items: 
    print(item)
    df_yes = full_df[full_df[item] == 1]
    df_no = full_df[full_df[item] == 0]
    
    yes_length_first, yes_length_second, yes_length_third = get_pronouns_per_row(df_yes)
    no_length_first, no_length_second, no_length_third = get_pronouns_per_row(df_no)
    
    print('yes first: ', yes_length_first, '\n', 'second: ', yes_length_second, '\n', 'third: ', yes_length_third)
    print('no first: ', no_length_first, '\n', 'second: ', no_length_second, '\n', 'third: ', no_length_third)

affcon_rapport
yes first:  5.426756283951754 
 second:  3.565049258815947 
 third:  1.2153577018690729
no first:  5.193611398368379 
 second:  3.4700677927151555 
 third:  1.1720096518441918
affcon_shareinformation
yes first:  5.353789529031774 
 second:  3.4965822879891832 
 third:  1.3051153008337715
no first:  5.4890767372927876 
 second:  3.7106158744099242 
 third:  0.8562959710176747
affcon_reasoning
yes first:  5.3607742712962265 
 second:  3.5493710928673643 
 third:  1.2239210665059876
no first:  5.3957962982327725 
 second:  3.46125692774234 
 third:  1.2862072571368817
affcon_gamemove
yes first:  5.35598937773631 
 second:  3.541950764372354 
 third:  1.2273020885667123
no first:  5.617977528089887 
 second:  3.4520102883443884 
 third:  1.2048192771084338


In [ ]:
### For full df 

In [61]:
import re 

for idx, row in full_df.iterrows():
    text = row['Input.full_text'].strip()
    text_split = text.split(' ')
    
    first_person_count = 0
    second_person_count = 0
    third_person_count = 0
    total_words = 0
    
    for word in text_split:
        word = re.sub("\.|\,|\:|\!|\?|\"|\'|\;|\)|\(", "", word)
        total_words += 1
        word = word.lower()
        
        if word in first_person: 
            first_person_count += 1
        elif word in second_person: 
            second_person_count += 1
        elif word in third_person:
            third_person_count += 1
            
        full_df.at[idx, 'first_person'] = float(first_person_count/ total_words)
        full_df.at[idx, 'second_person'] = float(second_person_count/ total_words)
        full_df.at[idx, 'third_person'] = float(third_person_count/ total_words)

In [62]:
df_no_rapport['first_person'].describe()

count    4420.000000
mean        0.053339
std         0.060171
min         0.000000
25%         0.000000
50%         0.043478
75%         0.095238
max         0.333333
Name: first_person, dtype: float64

In [63]:
df_no_rapport['second_person'].describe()

count    4420.000000
mean        0.037571
std         0.056038
min         0.000000
25%         0.000000
50%         0.000000
75%         0.071429
max         0.333333
Name: second_person, dtype: float64

In [64]:
df_no_rapport['third_person'].describe()

count    4420.000000
mean        0.010441
std         0.031292
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.333333
Name: third_person, dtype: float64

In [65]:
df_rapport['first_person'].describe()

count    11318.000000
mean         0.053954
std          0.057895
min          0.000000
25%          0.000000
50%          0.047619
75%          0.090909
max          0.333333
Name: first_person, dtype: float64

In [66]:
df_rapport['second_person'].describe()

count    11318.000000
mean         0.039349
std          0.053361
min          0.000000
25%          0.000000
50%          0.000000
75%          0.074074
max          0.400000
Name: second_person, dtype: float64

In [67]:
df_rapport['third_person'].describe()

count    11318.000000
mean         0.010886
std          0.030706
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.285714
Name: third_person, dtype: float64